### tf.data.Dataset으로 CSV 데이터를 로드하는 방법의 예를 제공. 

In [18]:
import functools

import numpy as np
import tensorflow as tf
import pandas as pd

In [19]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [20]:
np.set_printoptions(precision=3, suppress=True)

In [21]:
train_file_path

'/Users/jk/.keras/datasets/train.csv'

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...,...
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,0,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,1,female,19.0,0,0,30.0000,First,B,Southampton,y
625,0,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [31]:
LABEL_COLUMN = "survived"
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
        ignore_errors=True,
        **kwargs
    )
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [170]:
type(raw_train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [167]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print(f"{key:20s} : {value.numpy()}")

show_batch(raw_train_data)

sex                  : [b'female' b'male' b'male' b'male' b'male']
age                  : [40. 29. 22. 31. 17.]
n_siblings_spouses   : [1 0 0 0 0]
parch                : [1 0 0 0 2]
fare                 : [ 39.     30.      7.25   10.5   110.883]
class                : [b'Second' b'First' b'Third' b'Second' b'First']
deck                 : [b'unknown' b'D' b'unknown' b'unknown' b'C']
embark_town          : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone                : [b'n' b'y' b'y' b'y' b'n']


NoneType

In [ ]:
for a, b in raw_train_data.take(1):
    print(type(a))

In [33]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                  : [b'male' b'male' b'male' b'male' b'female']
age                  : [65. 44. 22. 32. 40.]
n_siblings_spouses   : [0 1 0 0 1]
parch                : [1 0 0 0 1]
fare                 : [ 61.979  26.      9.35    7.925 134.5  ]
class                : [b'First' b'Second' b'Third' b'Third' b'First']
deck                 : [b'B' b'unknown' b'unknown' b'unknown' b'E']
embark_town          : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone                : [b'n' b'n' b'y' b'y' b'n']


In [34]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                  : [30.5 47.  24.  28.   2. ]
n_siblings_spouses   : [0 1 2 0 4]
class                : [b'Third' b'Third' b'Second' b'First' b'Third']
deck                 : [b'unknown' b'unknown' b'unknown' b'A' b'unknown']
alone                : [b'y' b'n' b'n' b'y' b'n']


In [35]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS, column_defaults=DEFAULTS)

show_batch(temp_dataset)

age                  : [29. 23. 31. 28. 28.]
n_siblings_spouses   : [0. 0. 1. 0. 1.]
parch                : [0. 0. 0. 0. 1.]
fare                 : [ 9.5   13.792 18.     7.75  15.246]


In [37]:
example_batch, labels_batch = next(iter(temp_dataset))

In [38]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [39]:
packed_dataset = temp_dataset.map(pack)

# batch 5개, 각 col 4개
for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[ 51.      0.      0.     12.525]
 [ 32.5     1.      0.     30.071]
 [ 17.      0.      2.    110.883]
 [ 24.      0.      0.      7.496]
 [ 18.      0.      0.      7.796]]

[0 0 1 0 0]


In [40]:
example_batch, labels_batch = next(iter(temp_dataset))

In [47]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
    
    # 함수를 호출할 때 실행되는 매직 매서드
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features["numeric"] = numeric_features
        
        return features, labels
        

In [48]:
NUMERIC_FEATURES = ["age", "n_siblings_spouses", "parch", "fare"]

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [49]:
show_batch(packed_train_data)

sex                  : [b'male' b'female' b'female' b'male' b'male']
class                : [b'First' b'First' b'Second' b'Third' b'First']
deck                 : [b'C' b'B' b'unknown' b'unknown' b'D']
embark_town          : [b'Cherbourg' b'Cherbourg' b'Cherbourg' b'Southampton' b'Cherbourg']
alone                : [b'n' b'y' b'n' b'y' b'n']
numeric              : [[37.     0.     1.    29.7  ]
 [24.     0.     0.    69.3  ]
 [28.     1.     0.    24.   ]
 [16.     0.     0.     8.05 ]
 [23.     0.     1.    63.358]]


In [58]:
example_batch, labels_batch = next(iter(packed_train_data))

In [62]:
pd_data = pd.read_csv(train_file_path)
desc = pd_data[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [60]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [137]:
from sklearn.preprocessing import StandardScaler
def normalize_numeric_data(data, mean, std):
    # return (data-mean)/std
    # data가 eagerTensor 형태이므로 data.numpy()를 해야 값numpy 값을 얻을 수 있다.
#     print(type(sc.fit_transform(data.numpy())))
#     print(type(tf.convert_to_tensor(sc.fit_transform(data.numpy()))), tf.convert_to_tensor(sc.fit_transform(data.numpy())))
    sc = StandardScaler()
    return tf.convert_to_tensor(sc.fit_transform(data.numpy()))

In [138]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column("numeric", normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fa1de0d4d30>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [150]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 23.   ,   0.   ,   0.   ,  13.   ],
       [ 14.   ,   0.   ,   0.   ,   7.854],
       [ 38.   ,   0.   ,   0.   , 227.525],
       [ 24.   ,   1.   ,   0.   ,  26.   ],
       [ 57.   ,   0.   ,   0.   ,  12.35 ]], dtype=float32)>

In [152]:
type(example_batch)

collections.OrderedDict

In [140]:
desc.T['mean']

age                   29.631308
n_siblings_spouses     0.545455
parch                  0.379585
fare                  34.385399
Name: mean, dtype: float64

In [141]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(
[[-0.546 -0.5    0.    -0.52 ]
 [-1.146 -0.5    0.    -0.58 ]
 [ 0.453 -0.5    0.     1.995]
 [-0.48   2.     0.    -0.367]
 [ 1.719 -0.5    0.    -0.527]], shape=(5, 4), dtype=float32)


array([[-0.546, -0.5  ,  0.   , -0.52 ],
       [-1.146, -0.5  ,  0.   , -0.58 ],
       [ 0.453, -0.5  ,  0.   ,  1.995],
       [-0.48 ,  2.   ,  0.   , -0.367],
       [ 1.719, -0.5  ,  0.   , -0.527]], dtype=float32)

In [161]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [162]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [163]:
print(example_batch)
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

OrderedDict([('sex', <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'male', b'female', b'female', b'female', b'male'], dtype=object)>), ('class', <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Second', b'Third', b'First', b'Second', b'Second'], dtype=object)>), ('deck', <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'unknown', b'unknown', b'C', b'unknown', b'unknown'], dtype=object)>), ('embark_town', <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'Southampton', b'Southampton', b'Cherbourg', b'Southampton',
       b'Queenstown'], dtype=object)>), ('alone', <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'y', b'y', b'y', b'n', b'y'], dtype=object)>), ('numeric', <tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 23.   ,   0.   ,   0.   ,  13.   ],
       [ 14.   ,   0.   ,   0.   ,   7.854],
       [ 38.   ,   0.   ,   0.   , 227.525],
       [ 24.   ,   1.   ,   0.   ,  26.   ],
       [ 57.   ,   0.   ,   0.   ,  12.35 ]], dtype=float32)>)])
[1. 0. 0. 1

In [164]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [165]:
print(preprocessing_layer(example_batch).numpy()[0])

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(
[[-0.546 -0.5    0.    -0.52 ]
 [-1.146 -0.5    0.    -0.58 ]
 [ 0.453 -0.5    0.     1.995]
 [-0.48   2.     0.    -0.367]
 [ 1.719 -0.5    0.    -0.527]], shape=(5, 4), dtype=float32)
[ 1.     0.     0.     1.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     1.     0.    -0.546 -0.5
  0.    -0.52   1.     0.   ]
